## Factorization Machine

#### 1. Set-up
import dependent packages and declare consts

In [1]:
# package initialization

import pandas as pd
import numpy as np
from sklearn import metrics

from lightfm import LightFM
from lightfm.data import Dataset

In [2]:
# consts

DATASET = '../../data/extracted/LON-A/London_Attractions_Complete_Review.csv'
OCCURENCE_THRESHOLD = 5

USER_FEATURES = ['ugender', 'ucity', 'ucountry', 'ulevel']
ITEM_FEATURES = ['irating', 'itag']
# ITEM_FEATURES = ['iattribute', 'irating', 'itag', 'iprice']

#### 2. Read Dataset
read dataset in csv format into pandas.DataFrame

In [3]:
# read dataset

df = pd.read_csv(DATASET, sep='\t')

In [4]:
# print dataset information

print("Columns: \n", list(df.columns))
print("\nShape: \n", df.shape)

Columns: 
 ['Unnamed: 0', 'Unnamed: 0.1', 'iid', 'rid', 'rimages', 'rquote', 'rrate', 'rtime', 'uprofile', 'uage', 'ucity', 'ucountry', 'ugender', 'uhometown', 'uid_index', 'ulevel', 'uname_y', 'usince', 'ustate', 'ustyle', 'iattribute', 'ilocality', 'iname', 'ipopularity', 'ipost', 'irating', 'iregion', 'istreet', 'itag']

Shape: 
 (136978, 29)


#### 3. Data Preprocessing

* Retain users/items with at least five ratings only
* Data splitting
  - the latest 20% interactions (by time)
  - Randomly split the remaining data into training (70%) and validation (10%) sets
* Transform the ratings into binary implicit feedback as ground truth, indicating whether the user has interacted with the specific item

In [5]:
def sort_by_time(df):
    
    # here we use 'rid' for sorting becaz it's auto incrementing
    return df.sort_values(by=['rid'], ascending=True)

In [6]:
def filter_by_occurrence(df, column, threshold):
    return df.groupby(column).filter(lambda x: len(x) >= threshold)

In [7]:
def convert_binary(df):
    df.loc[df['rrate'] != "None", 'rrate'] = 1.0
    df.loc[df['rrate'] == "None", 'rrate'] = 0.0
    return df

In [8]:
def data_preprocess(dataframe):
    
    # sort by time (ascending order)
    df = sort_by_time(dataframe)
    
    # retrieve needed columns
    df = df[['uid_index', 'iid', 'rrate'] + USER_FEATURES + ITEM_FEATURES]
    
    # convert ratings into binarys
    df = convert_binary(df)
    
    df['rrate'] = pd.to_numeric(df['rrate'])
    
    # Retain users/items with at least five ratings only
    df = filter_by_occurrence(df, 'iid', OCCURENCE_THRESHOLD)
    df = filter_by_occurrence(df, 'uid_index', OCCURENCE_THRESHOLD)
    
    # split dataset into training set, validation set and test set
    users = df.groupby('uid_index')
    
    test_df = pd.DataFrame()
    train_validation_df = pd.DataFrame()
    
    # for each user, get its latest 20% rating as test set
    for uid in users.size().to_dict().keys():
        user = users.get_group(uid)
        split_idx = int(len(user)*0.8)
        test_df = test_df.append(user.iloc[split_idx:])
        train_validation_df = train_validation_df.append(user.iloc[:split_idx])
    
    train_validation_df = train_validation_df.reindex(np.random.permutation(train_validation_df.index)) # shuffle
    train_df = train_validation_df.iloc[:int(len(train_validation_df)*0.875)]
    validation_df = train_validation_df.iloc[int(len(train_validation_df)*0.875):]
    
    return (train_df, validation_df, test_df)

In [9]:
# dataset preprocessing

train_df, validation_df, test_df = data_preprocess(df)
print("training set size: ", train_df.shape)
print("validation set size: ", validation_df.shape)
print("test set size: ", test_df.shape)

training set size:  (90209, 9)
validation set size:  (12887, 9)
test set size:  (33178, 9)


#### 4. load into LightFM

In [11]:
# get list of all unique user ids
def get_unique_uids(df):
    return df['uid_index'].astype(str).unique()

# get user ids of each row in pandas.DataFrame
def get_uids(df):
    return df['uid_index'].astype(str).values

# get list of all unique user ids
def get_unique_iids(df):
    return df['iid'].astype(str).unique()

# get item ids of each row in pandas.DataFrame
def get_iids(df):
    return df['iid'].astype(str).values

# get list of all unique user features
def get_unique_user_features(df):
    sub_df = df[USER_FEATURES].astype(str)
    return np.unique(sub_df.values.flatten())

# get user features of each row in pandas.DataFrame
def get_user_features(df):
    sub_df = df[USER_FEATURES].astype(str)
    return sub_df.values

# get list of all unique item features
def get_unique_item_features(df):
    sub_df = df[ITEM_FEATURES].astype(str)
    return np.unique(sub_df.values.flatten())

# get item features of each row in pandas.DataFrame
def get_item_features(df):
    sub_df = df[ITEM_FEATURES].astype(str)
    return sub_df.values

# get list of tuples of user-item interactions
def get_iteractions(df, rating=False):
    
    if rating:
        columns = ['uid_index', 'iid', 'rrate']
    else:
        columns = ['uid_index', 'iid']
        
    df[['uid_index', 'iid']] = df[['uid_index', 'iid']].astype(str)
        
    rows = df[columns].values
    return (tuple(r) for r in rows)

In [20]:
print("# unique users:", len(get_unique_uids(train_df)))
print("# unique items:", len(get_unique_iids(train_df)))
print("# unique user features:", len(get_unique_user_features(train_df)))
print("# unique item features:", len(get_unique_item_features(train_df)))

# unique users: 16258
# unique items: 692
# unique user features: 3133
# unique item features: 628


create dataset

In [21]:
dataset = Dataset()

tell dataset what dimension of latent/user/item matrix to be used

In [22]:
dataset.fit(
    get_unique_uids(train_df),
    get_unique_iids(train_df),
    user_features = get_unique_user_features(train_df),
    item_features = get_unique_item_features(train_df)
)

In [23]:
num_users, num_items = dataset.interactions_shape()
print('Interaction matrix shape -- Num users: {} x num_items {}.'.format(num_users, num_items))

Interaction matrix shape -- Num users: 16258 x num_items 692.


build interaction matrix, which is the matrix storing user ratings

In [24]:
(interactions, weights) = dataset.build_interactions(get_iteractions(train_df, rating=True))

In [25]:
print(repr(interactions))
print(repr(weights))

<16258x692 sparse matrix of type '<class 'numpy.int32'>'
	with 90209 stored elements in COOrdinate format>
<16258x692 sparse matrix of type '<class 'numpy.float32'>'
	with 90209 stored elements in COOrdinate format>


build user feature matrix

In [26]:
uid_ufeats = ((uid, list(ufeats)) for uid, ufeats in zip(get_uids(train_df), get_user_features(train_df)) )

user_features = dataset.build_user_features(uid_ufeats)
print(repr(user_features))

<16258x19391 sparse matrix of type '<class 'numpy.float32'>'
	with 76750 stored elements in Compressed Sparse Row format>


build item feature matrix

In [27]:
iid_ifeats = ((iid, list(ifeats)) for iid, ifeats in zip(get_iids(train_df), get_item_features(train_df)) )

item_features = dataset.build_item_features(iid_ifeats)
print(repr(item_features))

<692x1320 sparse matrix of type '<class 'numpy.float32'>'
	with 2076 stored elements in Compressed Sparse Row format>


#### 5. Model declaratoin & fitting

to tune model parameters, please check [lightfm.LightFM()](https://making.lyst.com/lightfm/docs/lightfm.html)

In [28]:
# consts for model & training

LOSS = 'logistic'
EPOCH = 30

In [29]:
model = LightFM(loss=LOSS)
model.fit(interactions, item_features=item_features, user_features=user_features, epochs=EPOCH)

#### 6. Prediction

In [30]:
def predict(model, dataframe):
    df = dataframe[['uid_index', 'iid', 'rrate']]
    z = []
    y = []
    for i in range(len(df)):
        user = df.iloc[i][0]
        item = df.iloc[i][1]
        rating = df.iloc[i][2]
        prediction = model.predict([user], [item])
        
        z.append(prediction[0])
        y.append(rating)
    return (np.array(z, dtype=np.float32), np.array(y, dtype=np.int))

In [31]:
train_df

,uid_index,iid,rrate,ugender,ucity,ucountry,ulevel,irating,itag
42135,13927,4,0.0,female,Navan,Ireland,Level 4 Contributor,4.5,"['clock tower', 'elizabeth tower', 'london eye..."
83291,4724,307,1.0,NaN,England,United Kingdom,Level 4 Contributor,4.5,"['swan lake', 'covent garden', 'backstage tour..."
75746,9162,9,1.0,male,Birmingham,United Kingdom,Level 5 Contributor,4.5,"['winter wonderland', 'speakers corner', 'serp..."
47466,11667,221,1.0,NaN,NaN,NaN,Level 3 Contributor,3.5,"['westminster palace', 'original palace', 'eng..."
93362,13570,314,1.0,female,Derby,United Kingdom,Level 4 Contributor,4.5,"['audrey hepburn', 'modern portraits', 'taylor..."
...,...,...,...,...,...,...,...,...,...
18760,9915,24,1.0,NaN,Draper,United States,Level 6 Contributor,4.5,"['street performers', 'market stalls', 'transp..."
13238,13299,626,1.0,male,London,United Kingdom,Level 6 Contributor,4.5,"['national theatre', 'london eye', 'street per..."
65120,15955,7,1.0,NaN,Manchester,United Kingdom,Level 4 Contributor,4.5,"['guided tour', 'westminster hall', 'local mp'..."
26512,14286,317,1.0,NaN,Labuan Federal Territory,Malaysia,Level 6 Contributor,4.5,"['viewing platform', 'glass of champagne', 'th..."


In [32]:
validation_z, validation_y = predict(model, validation_df)

In [33]:
validation_z = (validation_z - np.min(validation_z)) / np.ptp(validation_z)

In [34]:
test_z, test_y = predict(model, test_df)

In [35]:
test_z = (test_z - np.min(test_z)) / np.ptp(test_z)

#### 7. Evaluation

AUC metric

In [36]:
def evaluate_auc(z, y):
    return metrics.roc_auc_score(y, z)

In [37]:
print("validation AUC: ", evaluate_auc(validation_z, validation_y))
print("test AUC: ", evaluate_auc(test_z, test_y))

validation AUC:  0.5153114590031755
test AUC:  0.5103611831544499


LogLoss metric

In [38]:
# assume parameters z & y are ndarray
def evaluate_logloss(z, y):
    zz = np.ones((z.shape[0], 2))
    zz[:, 0] -= z
    zz[:, 1] = z
    return metrics.log_loss(y, zz)

In [39]:
print("validation LogLoss: ", evaluate_logloss(validation_z, validation_y))
print("test LogLoss: ", evaluate_logloss(test_z, test_y))

validation LogLoss:  3.5314239833549883
test LogLoss:  4.112882389394414


NDCG metric

In [40]:
# assume parameters z & y are ndarray
def evaluate_ndcg(z, y):
    return metrics.ndcg_score(np.expand_dims(y, axis=0), np.expand_dims(z, axis=0), k=5)

In [41]:
print("validation NDCG@5: ", evaluate_ndcg(validation_z, validation_y))
print("test NDCG@5: ", evaluate_ndcg(test_z, test_y))

validation NDCG@5:  0.9999999999999999
test NDCG@5:  0.44685352999185624
